In [3]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "dbuser"
password = "devin1"

# Connect to database via CRUD Module
db = AnimalShelter()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = 'Grazioso.png'  # Replace with your own image path
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='customer image'),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Devin Criswell Project 2 Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[],
        data=[],
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",  # Updated: choose "single" or "multi"
        row_selectable="single",  # Updated: choose "single" or "multi"
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on the selected filter type
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Read the data from the database based on the filter
    if filter_type == 'water':
        data = db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'mount':
        data = db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'disaster':
        data = db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        })
    else:
        data = db.read({})

    # Convert ObjectId to string
    for record in data:
        if '_id' in record:
            record['_id'] = str(record['_id'])

    # Define the columns for the data table
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in data[0].keys()] if data else []

    # Return the data and columns to the data table
    return data, columns
# Callback to generate a graph based on the selected data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData:
        dff = pd.DataFrame.from_dict(viewData)
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Preferred Animals')
            )
        ]
    return []

# Callback to highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the map based on the selected data
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or not viewData:
        return []  # Return empty if there's no data

    dff = pd.DataFrame.from_dict(viewData)

    # Check if index is within the bounds of the DataFrame
    if len(dff) == 0 or index[0] >= len(dff):
        return []  # Return empty if index is out of bounds

    # Handle single row selection
    row = index[0]  # This is safe now as we've checked the bounds
    
    # Austin TX coordinates for the map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Run the server
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8251/
